In [6]:
import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
import re
from typing import Tuple
from sklearn.preprocessing import LabelEncoder

In [3]:
def load_file(file_path: str) -> Tuple[Tuple[str], Tuple[str]]:
    """loads the test file and extracts all functions/derivatives"""
    data = open(file_path, "r").readlines()
    functions, derivatives = zip(*[line.strip().split("=") for line in data])
    return functions, derivatives

In [19]:
max_sequence_len = 30
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 990000  # Number of samples to train on.
# Path to the data txt file on disk.
filepath = "sample_train.txt"

# functions, true_derivatives = load_file(filepath)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(filepath, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
random.shuffle(lines)
with open('sample_test.txt', 'w') as f:
    f.write('\n'.join(lines[num_samples+1:]))
with open('sample_train.txt', 'w') as f:
    f.write('\n'.join(lines[:num_samples]))
lines = lines[:num_samples]
print(len(lines))

In [39]:
functions = []
derivatives = []
input_characters = set()
# target_characters = set()
with open(filepath, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in list(filter(None,lines[: min(num_samples, len(lines) - 1)])):
    function, derivative = line.split("=")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    derivative = "\t" + derivative + "\n"
    functions.append(function)
    derivatives.append(derivative)
    for char in list(filter(None, re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#)", function))):
        if char not in input_characters:
            input_characters.add(char)
    # for char in target_text:
    #     if char not in target_characters:
    #         target_characters.add(char)
input_characters.add('\n')
input_characters.add('\t')
input_characters.add('')
input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(functions))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_encoder_tokens)
print("Max sequence length for inputs:", max_sequence_len)
print("Max sequence length for outputs:", max_sequence_len)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


Number of samples: 989998
Number of unique input tokens: 43
Number of unique output tokens: 43
Max sequence length for inputs: 30
Max sequence length for outputs: 30


In [40]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)

for i, (function, derivative) in enumerate(zip(functions[:5], derivatives[:5])):
    for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#)", function)))):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index['']] = 1.0
    for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#)", derivative)))):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, input_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index['']] = 1.0
    decoder_target_data[i, t:, input_token_index['']] = 1.0



In [41]:
"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_encoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


2022-09-10 22:18:40.384061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
"""
## Train the model
"""

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100
  126/12375 [..............................] - ETA: 2:53:34 - loss: 0.0000e+00 - accuracy: 0.0079

KeyboardInterrupt: 

In [ ]:
"""
## Run inference (sampling)
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in input_token_index.items())


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_encoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, input_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_sequence_len:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_encoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


"""
You can now generate decoded sentences as such:
"""

for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", functions[seq_index])
    print("Decoded sentence:", decoded_sentence)
